In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))
spark = SparkSession.builder.getOrCreate()

24/03/17 10:21:29 WARN Utils: Your hostname, Rezaei-MR resolves to a loopback address: 127.0.1.1; using 172.26.205.103 instead (on interface eth0)
24/03/17 10:21:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/17 10:21:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/17 10:21:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.csv('../data/greenhouse-gas-emissions-by-region-industry-and-ho.csv', header=True)

In [3]:
def pretty_print(dataframe, max_lenght=20, **kwargs):
    import math
    max_lenght = max_lenght
    for col in dataframe.columns:
        if len(col) > max_lenght:
            max_lenght = len(col)
    for col in dataframe.columns:
        preffix = math.floor((max_lenght - len(col)) / 2)
        suffix = math.ceil((max_lenght - len(col)) / 2)
        new_name = " " * preffix + col + " " * suffix
        dataframe = dataframe.withColumnRenamed(col, new_name)
    dataframe.show(**kwargs)
    del(dataframe)

In [4]:
pretty_print(df)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       region       | anzsic_descriptor  |    sub_industry    | household_category |        gas         |       units        |     magnitude      |        year        |      data_val      |
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|            Auckland|Forestry and logg...|               Total|               Total|Carbon dioxide eq...|          Kilotonnes|Carbon dioxide eq...|                2007|               45.92|
|            Auckland|Forestry and logg...|               Total|               Total|Carbon dioxide eq...|          Kilotonnes|Carbon dioxide eq...|                2008|               45.88|
|            Auckland|Forestry and logg...|  

In [5]:
def low_mem_spark_groupby_with_multiple_aggregates(df, groupby_col, agg_cols=[], agg_funcs=[], max_cols_per_partition=2):
    dfs = []
    agg_names = []
    for i in range(len(agg_cols)):
        agg_names.append(agg_funcs[i].__name__ + "(" + agg_cols[i] + ")")
    for i in range(0, len(agg_cols), max_cols_per_partition):
        j = min(i + max_cols_per_partition, len(agg_cols) - 1)
        dfs.append(df.groupby(groupby_col).agg(
            *(agg_funcs[k](agg_cols[k]).alias(agg_funcs[k].__name__ + "(" + agg_cols[k] + ")") for k in range(i, j + 1))
        ))
    ret = dfs[0]
    for d in dfs[1:]:
        ret = ret.join(d, groupby_col)
    ret.cache()
    del dfs
    return ret

In [6]:
summary_df = low_mem_spark_groupby_with_multiple_aggregates(df, "region",
                                               agg_cols=["anzsic_descriptor", "sub_industry", "household_category", "gas", "units",
                                                         "magnitude", "year", "year", "year", "data_val", "data_val", "data_val", "data_val", "data_val", "data_val"],
                                               agg_funcs=[F.first, F.first, F.first, F.first, F.first, F.first, F.min, F.max, F.avg, F.sum, F.variance, F.stddev, F.min, F.mode, F.avg, F.max],
                                               max_cols_per_partition=2)
pretty_print(summary_df)

+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
|         region          |first(anzsic_descriptor) |   first(sub_industry)   |first(household_category)|first(household_category)|       first(gas)        |      first(units)       |      first(units)       |    first(magnitude)     |        min(year)        |        min(year)        |        max(year)        |        avg(year)        |        avg(year)        |      sum(data_val)      |   varian